In [10]:
import os
import sys
from pathlib import Path
import sqlite3
import json
from jsonschema import Draft4Validator

In [11]:
with open('schema.json') as f:
    schema=json.load(f)

validator=Draft4Validator(schema)

In [12]:
#set directories
root_dir=Path('/home/samer/projects/fuzzy_sql')
metadata_dir=os.path.join(root_dir,'data/cms/processed/metadata')
db_path=os.path.join(root_dir,'db/cms.db')

In [13]:
#define subject table names in the database
tbl_names_lst=['s1_ben_sum_2008','s1_ben_sum_2009','s1_ben_sum_2010','s1_carrier_1a','s1_carrier_1b','s1_inpatient','s1_outpatient','s1_prescrp']

In [14]:
#Generate default metadata json files from table info in db
from genericpath import isfile


con=sqlite3.connect(db_path)
cur = con.cursor()

for tbl_name in tbl_names_lst:
    metadata={}
    metadata['table_name']=tbl_name

    cur.execute(f"PRAGMA table_info({tbl_name})")
    _,var_name, var_type,_,_,_=zip(*cur.fetchall())
    var_tpls=list(zip(var_name,var_type))
    var_lst=[[var_tpl[0],var_tpl[1]] for var_tpl in var_tpls]
    metadata['table_vars']=var_lst

    parent_details={
        "parent1":[
            ["parent1_key"],["self1_key"]
            ],
        "parent2":[
            ["parent1_key"],["self1_key"]  
        ],
        "parent3":[
            ["parent3_key"],["self3_key"]  
        ],
    }
    metadata['parent_details']=parent_details

    fname=os.path.join(metadata_dir,tbl_name+".json")
    if os.path.isfile(fname):
        ans=input('Do you really want to replace the existing JSON metadata file? (y/n)')
        if ans=='n':
            continue
    with open(fname, "w") as outfile:
        json.dump(metadata, outfile)